# Análisis del Mercado Eléctrico Italiano

## Contexto
Este análisis se basa en un conjunto de datos sobre el mercado eléctrico italiano, disponible en Kaggle. El dataset cubre el periodo de 2012 a 2022 y proporciona información sobre la demanda de electricidad, las fuentes de producción de energía y los precios de la electricidad en Italia. Durante este periodo, Italia ha enfrentado una transición energética debido a cambios en la oferta y la demanda de energía, impulsados por factores como las interrupciones en el suministro de gas, la mayor inversión en energías renovables y las tensiones geopolíticas. Estos cambios son cruciales para comprender las variaciones en la demanda de electricidad.

El objetivo de este análisis es explorar y entender cómo ha evolucionado la demanda eléctrica a lo largo de los años, comparando las distintas regiones del país y realizando predicciones sobre la futura demanda. Este análisis es clave para la optimización de la producción y distribución de energía.

## Archivos del Dataset
El conjunto de datos está compuesto por varios archivos CSV que contienen información específica sobre la producción, la demanda y los precios de la electricidad en Italia. A continuación se describen los archivos y sus características:

1. **`NYMEX_DL_TTF1_1D.csv`** (73.38 KB)
   - **Descripción**: Contiene datos sobre los futuros continuos del gas natural holandés (Dutch Natural Gas Continuous Future), que es relevante para entender los precios de la energía en Europa.
   - **Características**: Datos de precios futuros del gas natural, lo cual puede influir en el precio de la electricidad.

2. **`electricity-prod-source-stacked.csv`** (377.27 KB)
   - **Descripción**: Proporciona información sobre la producción de electricidad en Italia, desglosada por fuente de energía (renovables, fósiles, etc.).
   - **Características**: Contiene la proporción de electricidad generada por cada tipo de fuente de energía, lo que es esencial para comprender cómo varía la oferta de electricidad según las fuentes de energía.

3. **`it_electricity_demand_adj.csv`** (7.94 MB)
   - **Descripción**: Contiene datos ajustados sobre la demanda eléctrica horaria en Italia durante el período cubierto (2012-2022).
   - **Características**: Detalla el consumo de electricidad por hora en diferentes regiones de Italia (por ejemplo, Norte, Sur, Centro-Norte, etc.).

4. **`it_electricity_prices_adj.csv`** (11.97 MB)
   - **Descripción**: Contiene información sobre los precios horarios de la electricidad en Italia, ajustados para reflejar las tendencias económicas.
   - **Características**: Información sobre el precio de la electricidad por megavatio hora (€/MWh), lo que permite observar las fluctuaciones de los precios a lo largo del tiempo.

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Cargar los datos
demanda_electrica_horaria_it_df = pd.read_csv('it_electricity_demand_adj.csv')
prices_electricidad_ph_df = pd.read_csv('it_electricity_prices_adj.csv')
prod_electricidad__df = pd.read_csv('electricity-prod-source-stacked.csv')
gas_precios_futuros_df = pd.read_csv('NYMEX_DL_TTF1 1D.csv')



### EMPEZAMOS A LIMPIAR Y ANALIZAR EL PRIMER DATASET

In [57]:
print('Info dataset demanda electrica horaria en Italia desde 2012 a 2022:')
print(demanda_electrica_horaria_it_df.info())

Info dataset demanda electrica horaria en Italia desde 2012 a 2022:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93504 entries, 0 to 93503
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  93504 non-null  object 
 1   Italia      93504 non-null  float64
 2   CNOR        93504 non-null  float64
 3   CSUD        93504 non-null  float64
 4   NORD        93504 non-null  float64
 5   SARD        93504 non-null  float64
 6   SICI        93504 non-null  float64
 7   SUD         93504 non-null  float64
dtypes: float64(7), object(1)
memory usage: 5.7+ MB
None


In [58]:
# Convertir la columna 'Unnamed: 0' a formato de fecha y hora
demanda_electrica_horaria_it_df['Unnamed: 0'] = pd.to_datetime(demanda_electrica_horaria_it_df['Unnamed: 0'])

# renombrar la columna "Unnamed" a "datetime"
demanda_electrica_horaria_it_df = demanda_electrica_horaria_it_df.rename(columns={'Unnamed: 0': 'Datetime'})

# Verificar la conversión y la estructura de los datos
print(demanda_electrica_horaria_it_df.head())




             Datetime     Italia      CNOR      CSUD       NORD      SARD  \
0 2012-01-01 01:00:00  24659.161  2662.703  4534.718  11693.113  1460.687   
1 2012-01-01 02:00:00  23387.590  2539.670  4240.313  11113.677  1437.934   
2 2012-01-01 03:00:00  22090.760  2403.112  3927.174  10558.286  1410.237   
3 2012-01-01 04:00:00  21009.116  2267.737  3692.188  10126.168  1379.016   
4 2012-01-01 05:00:00  20350.988  2186.529  3569.043   9851.236  1340.877   

       SICI       SUD  
0  2013.582  2294.358  
1  1926.623  2129.373  
2  1825.260  1966.691  
3  1712.619  1831.388  
4  1627.099  1776.204  


In [59]:
# Comprobar si hay valores nulos en demanda_electrica_horaria_it_df
print('VALORES NULOS:')
print(demanda_electrica_horaria_it_df.isnull().sum())
print('-----------------------------------------------------------------------------------')
print(demanda_electrica_horaria_it_df.describe())

VALORES NULOS:
Datetime    0
Italia      0
CNOR        0
CSUD        0
NORD        0
SARD        0
SICI        0
SUD         0
dtype: int64
-----------------------------------------------------------------------------------
                  Datetime        Italia          CNOR          CSUD  \
count                93504  93504.000000  93504.000000  93504.000000   
mean   2017-05-02 00:30:00  32519.967098   3287.953532   5247.286205   
min    2012-01-01 01:00:00  16272.977000   1550.070000   2580.443000   
25%    2014-09-01 00:45:00  26502.196750   2643.802000   4318.080750   
50%    2017-05-02 00:30:00  32016.382500   3224.852000   5258.185500   
75%    2020-01-01 00:15:00  38450.967750   3913.896750   6099.085000   
max    2022-09-01 00:00:00  50393.780000   6007.450000   8899.763000   
std                    NaN   6818.933965    791.864286   1128.337246   

               NORD          SARD          SICI           SUD  
count  93504.000000  93504.000000  93504.000000  93504.000000  

In [68]:
# funciones para definir estación y momento del día
def asignar_estacion_en_ingles(mes):
    if mes in [12, 1, 2]:
        return 'Winter'
    elif mes in [3, 4, 5]:
        return 'Spring'
    elif mes in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'
    
def asignar_momento_dia_en_ingles(hora):
    if hora >= 0 and hora < 6:
        return 'Night'
    elif hora >= 6 and hora < 12:
        return 'Morning'
    elif hora >= 12 and hora < 18:
        return 'Afternoon'    
    else:
        return 'Evening'
    
# Añadir las columnas 'estaciòn', 'momento del dia', 'año' y 'mes' al dataframe.
demanda_electrica_horaria_it_df['year'] = demanda_electrica_horaria_it_df['Datetime'].dt.year
demanda_electrica_horaria_it_df['month'] = demanda_electrica_horaria_it_df['Datetime'].dt.month_name()
demanda_electrica_horaria_it_df['season'] = demanda_electrica_horaria_it_df['Datetime'].dt.month.apply(asignar_estacion_en_ingles)
demanda_electrica_horaria_it_df['time_of_day'] = demanda_electrica_horaria_it_df['Datetime'].dt.hour.apply(asignar_momento_dia_en_ingles)

print(demanda_electrica_horaria_it_df)


                 Datetime     Italia      CNOR      CSUD       NORD      SARD  \
0     2012-01-01 01:00:00  24659.161  2662.703  4534.718  11693.113  1460.687   
1     2012-01-01 02:00:00  23387.590  2539.670  4240.313  11113.677  1437.934   
2     2012-01-01 03:00:00  22090.760  2403.112  3927.174  10558.286  1410.237   
3     2012-01-01 04:00:00  21009.116  2267.737  3692.188  10126.168  1379.016   
4     2012-01-01 05:00:00  20350.988  2186.529  3569.043   9851.236  1340.877   
...                   ...        ...       ...       ...        ...       ...   
93499 2022-08-31 20:00:00  39234.945  3450.666  6835.219  21600.813  1233.899   
93500 2022-08-31 21:00:00  39005.320  3417.592  6790.829  21280.509  1260.336   
93501 2022-08-31 22:00:00  38956.535  3358.284  6855.084  20964.329  1305.351   
93502 2022-08-31 23:00:00  37077.835  3172.588  6478.296  20155.819  1250.641   
93503 2022-09-01 00:00:00  34155.930  2868.968  5940.663  18603.728  1162.996   

           SICI       SUD  

In [69]:
# Crear una lista de las columnas para las que queremos analizar la demanda
columnas_demandas = ['Italia', 'CNOR', 'CSUD', 'NORD', 'SARD', 'SICI', 'SUD']

# Agrupar por 'year', 'time_of_day' y 'zona' (las columnas de demanda) y calcular la demanda total
demanda_agrupada = demanda_electrica_horaria_it_df.groupby(['year', 'time_of_day'])[columnas_demandas].sum()

# Mostrar el resultado
print(demanda_agrupada)

                        Italia         CNOR          CSUD          NORD  \
year time_of_day                                                          
2012 Afternoon    8.089692e+07  8715777.574  1.311165e+07  4.398976e+07   
     Evening      7.987270e+07  8662182.291  1.363655e+07  4.112202e+07   
     Morning      7.616938e+07  8122821.775  1.181712e+07  4.200404e+07   
     Night        5.759578e+07  6078902.160  9.109768e+06  3.059071e+07   
2013 Afternoon    7.874992e+07  7823442.493  1.225968e+07  4.389973e+07   
     Evening      7.734604e+07  8102652.919  1.283570e+07  4.059038e+07   
     Morning      7.375730e+07  7276235.247  1.094626e+07  4.162573e+07   
     Night        5.534996e+07  5315410.365  8.294864e+06  3.031154e+07   
2014 Afternoon    7.591198e+07  6939186.971  1.091580e+07  4.362234e+07   
     Evening      7.616782e+07  7497878.166  1.208861e+07  4.078020e+07   
     Morning      7.179528e+07  6556670.054  9.882450e+06  4.163845e+07   
     Night        5.45865

In [64]:
demand_por_zona_por_anio = demanda_electrica_horaria_it_df.groupby(['year']).agg({
    'Italia': 'sum',  # Sumar la columna 'Italia'
    'CNOR': 'sum',    # Sumar la columna 'CNOR'
    'CSUD': 'sum',    # Sumar la columna 'CSUD'
    'NORD': 'sum',    # Sumar la columna 'NORD'
    'SARD': 'sum',    # Sumar la columna 'SARD'
    'SICI': 'sum',    # Sumar la columna 'SICI'
    'SUD': 'sum'
}).reset_index()

print(demand_por_zona_por_anio)

    year        Italia          CNOR          CSUD          NORD  \
0   2012  2.945348e+08  3.157968e+07  4.767508e+07  1.577065e+08   
1   2013  2.852032e+08  2.851774e+07  4.433649e+07  1.564274e+08   
2   2014  2.784616e+08  2.586894e+07  4.064183e+07  1.565127e+08   
3   2015  2.826228e+08  2.811454e+07  4.497811e+07  1.558118e+08   
4   2016  2.822920e+08  2.985886e+07  4.615561e+07  1.552723e+08   
5   2017  2.860419e+08  3.124860e+07  4.657017e+07  1.591522e+08   
6   2018  2.917606e+08  3.108157e+07  4.594104e+07  1.644496e+08   
7   2019  2.889769e+08  3.095200e+07  4.574050e+07  1.620192e+08   
8   2020  2.715093e+08  2.890833e+07  4.384289e+07  1.507080e+08   
9   2021  2.860881e+08  2.457109e+07  5.061676e+07  1.609361e+08   
10  2022  1.932558e+08  1.673546e+07  3.414375e+07  1.082007e+08   

            SARD          SICI           SUD  
0   1.263108e+07  1.991133e+07  2.503108e+07  
1   1.089503e+07  1.932450e+07  2.570209e+07  
2   1.141631e+07  1.804945e+07  2.597243e+

# secondo dataset

In [ ]:
# print('------------------------------------------------------------------------------------------------------------------------------')
# print('Info dataset precios horarios electricidad en Italia:')
# print(prices_electricidad_ph_df.info())
# print('------------------------------------------------------------------------------------------------------------------------------')
# print('Info dataset producción energia electrica en Italia por fuente de energia:')
# print(prod_electricidad__df.info())
# print('------------------------------------------------------------------------------------------------------------------------------')
# print('Info dataset precios futuros del gas natural en Europa:')
# print(gas_precios_futuros_df.info())